In [37]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline


In [38]:
# df = pd.read_csv('../data/analyst_ratings_processed_final.csv')[['headline', 'stock', 'headline_cleaned']]
df = pd.read_csv('../data/analyst_ratings_processed_final.csv')
# df = df.head(500000)
# df.sample()

,id,headline,url,publisher,date,stock,headline_cleaned,senti_pos_score,senti_neg_score,senti_score,open,close
47166,410760,8x8 shares are trading higher after the company announced the appointment of 3 new executives that are expected to build on market success.,https://www.benzinga.com/wiim/20/01/15137211/8x8-shares-are-trading-higher-after-the-company-announced-the-appointment-of-3-new-executives-that-a,Benzinga Newsdesk,2020-01-16,EGHT,"['share.n.01', 'trade.v.01', 'high.a.01', 'company.n.01', 'announce.v.01', 'appointment.n.01', 'new.a.01', 'executive.n.01', 'expect.v.01', 'construct.v.01', 'market.n.01', 'success.n.01']",1.125,0.5,0.625,20.29,21.28


In [39]:
def stockDevelopement(row):
    stock_differenz = row['close'] - row['open']
    if stock_differenz < 0:
        return -1
    elif stock_differenz > 0:
        return 1
    else: return 0

In [40]:
df['stockPriceChange'] = df.apply(stockDevelopement, axis=1)

In [41]:
df = df[['id','headline', 'stock', 'headline_cleaned','open','close', "stockPriceChange"]]
df = df.rename(columns={'headline': '_headline', 'headline_cleaned':'_headline_cleaned', 'stock': '_stock', 'open': '_open', 'close':'_close', 'stockPrice_Change':'_stockPrice_Change'})

In [42]:
df = df.set_index('id')
df = df.sort_index()

df.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange
id,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,1
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,1
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,1
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', 'session.n.01']",85.00,84.98,1
4,"B of A Securities Maintains Neutral on Agilent Technologies, Raises Price Target to $88",A,"['security.n.01', 'neutral.n.01', 'technology.n.01', 'raise.n.01', 'monetary_value.n.01', 'target.n.01']",85.00,84.98,1


In [43]:
def tokenz_in_sentence(hl):
    word_list = hl[1:-1].split(", ")
    word_list = [word[1:-6] for word in word_list]
    sentence = ' '.join(word_list)
    return sentence

In [44]:
df['_headline_sentence'] = df._headline_cleaned.apply(tokenz_in_sentence)

In [45]:
df_set = df['_headline_sentence'].tolist()

In [46]:
df.shape

(50000, 7)

In [47]:
count_vectorizer = CountVectorizer(analyzer="word", stop_words="english")
tfidf_vectorizer = TfidfVectorizer(analyzer='word', stop_words='english', smooth_idf=True, use_idf=True)

count_wm = count_vectorizer.fit_transform(df_set)
tfidf_wm = tfidf_vectorizer.fit_transform(df_set)

count_tokenz = count_vectorizer.get_feature_names()
tfidf_tokenz = tfidf_vectorizer.get_feature_names()

df_countvect = pd.DataFrame(data=count_wm.toarray(), index=df.index, columns=count_tokenz)
df_tfidfvect = pd.DataFrame(data= tfidf_wm.toarray(), index=df.index, columns=tfidf_tokenz)

In [48]:
# # export the Matrixes because of some Allocation error sometimes
# df_countvect.to_csv('../data/df_countvect.csv', index=False)
# df_tfidfvect.to_csv('../data/df_tfidfvect.csv', index=False)

In [49]:
cols_to_delete = []
for column in df_tfidfvect:
    count = df_tfidfvect[df_tfidfvect[column] > 0].shape[0]
    if count<5:
        cols_to_delete.append(column)

print(len(cols_to_delete))
print(df_tfidfvect.shape[1])

3020
5686


In [50]:
df_tfidfvect.drop(axis=1, columns=cols_to_delete, inplace=True)

In [51]:
# mergred = pd.read_csv('../data/analyst_ratings_processed_final_tfidf_merged.csv')#
mergred =df.merge(df_tfidfvect, on='id')

In [52]:
pd.set_option('display.max_colwidth', None)

mergred.head(2)

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence,aare,aaron,ability,...,writer,wuhan,xerox,yak,year,yesterday,yip,york,yuan,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,1,stock hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,1,stock hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [53]:
import numpy as np

In [54]:
from sklearn.model_selection import RandomizedSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 50, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 3]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [30, 37, 45, 53, 61, 68, 76, 84, 92, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, None], 'min_samples_split': [2, 3], 'min_samples_leaf': [1, 2], 'bootstrap': [True, False]}


In [55]:
from sklearn.ensemble import RandomForestRegressor


In [56]:
# rf = RandomForestRegressor()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 20, cv = 3, verbose=2)

In [57]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence,aare,aaron,ability,...,writer,wuhan,xerox,yak,year,yesterday,yip,york,yuan,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,1,stock hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,1,stock hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,1,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', 'session.n.01']",85.00,84.98,1,stock travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"B of A Securities Maintains Neutral on Agilent Technologies, Raises Price Target to $88",A,"['security.n.01', 'neutral.n.01', 'technology.n.01', 'raise.n.01', 'monetary_value.n.01', 'target.n.01']",85.00,84.98,1,security neutral technology raise monetary_value target,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
mergred.head()

,_headline,_stock,_headline_cleaned,_open,_close,stockPriceChange,_headline_sentence,aare,aaron,ability,...,writer,wuhan,xerox,yak,year,yesterday,yip,york,yuan,zone
id,,,,,,,,,,,,,,,,,,,,,
0,Stocks That Hit 52-Week Highs On Friday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'friday.n.01']",92.13,90.38,1,stock hit high friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stocks That Hit 52-Week Highs On Wednesday,A,"['stock.n.01', 'hit.v.01', 'high.n.01', 'wednesday.n.01']",90.65,90.49,1,stock hit high wednesday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,71 Biggest Movers From Friday,A,"['large.a.01', 'mover.n.01', 'friday.n.01']",86.23,86.13,1,large mover friday,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,46 Stocks Moving In Friday's Mid-Day Session,A,"['stock.n.01', 'travel.v.01', 'friday.n.01', 'session.n.01']",85.00,84.98,1,stock travel friday session,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"B of A Securities Maintains Neutral on Agilent Technologies, Raises Price Target to $88",A,"['security.n.01', 'neutral.n.01', 'technology.n.01', 'raise.n.01', 'monetary_value.n.01', 'target.n.01']",85.00,84.98,1,security neutral technology raise monetary_value target,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
X = mergred.drop(columns=['_headline', '_stock', '_headline_cleaned', '_open', '_close', 'stockPriceChange', '_headline_sentence'])
y= mergred['stockPriceChange']

In [60]:
# Fit the random search model
# rf_random.fit(X, y)


In [61]:
# rf_random.best_params_

In [62]:
from sklearn.model_selection import train_test_split

In [63]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=0)

In [64]:
feature_words = list(X_train)
dflist = pd.DataFrame(feature_words)
print(dflist)
dflist.to_csv('../data/tfidf_features.csv', index=False)

                0
0            aare
1           aaron
2         ability
3            able
4     abstraction
...           ...
2661    yesterday
2662          yip
2663         york
2664         yuan
2665         zone

[2666 rows x 1 columns]


In [65]:
from sklearn.ensemble import RandomForestRegressor

regressor = RandomForestRegressor( n_estimators=37, min_samples_split=2, min_samples_leaf=2, max_features='sqrt', max_depth=10, bootstrap=True)
regressor.fit(X_train, y_train,)
y_pred = regressor.predict(X_test)

In [66]:
y_pred_bin = []
for value in y_pred:
    if value < 1:
        y_pred_bin.append(-1)
    elif value > 1:
        y_pred_bin.append(1)
    else: y_pred_bin.append(0)

In [67]:
from sklearn import metrics
import numpy as np

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred_bin))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred_bin))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred_bin)))

Mean Absolute Error: 0.9984
Mean Squared Error: 1.988
Root Mean Squared Error: 1.4099645385611654


In [68]:
print('Accuracy Score:', metrics.accuracy_score(y_test, y_pred_bin))

Accuracy Score: 0.4964


In [69]:
pd.read_csv('../data/tfidf_features.csv')['0']

0              aare
1             aaron
2           ability
3              able
4       abstraction
           ...     
2661      yesterday
2662            yip
2663           york
2664           yuan
2665           zone
Name: 0, Length: 2666, dtype: object

In [70]:
def predictHeadline(hl):

    count_hl = len(list(hl.split()))
    single_headline_tfidf = pd.DataFrame()
    feature_list = list(X)
    for feature in feature_list:
        idf_value = tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[feature]]
        count_feature = hl.count(feature)
        single_headline_tfidf[feature] = [(count_feature / count_hl) * idf_value]
    print(regressor.predict(single_headline_tfidf))
    
predictHeadline("stock very high best apple low")


C:\Users\kaste\AppData\Local\Temp/ipykernel_17932/1561814447.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  single_headline_tfidf[feature] = [(count_feature / count_hl) * idf_value]


[0.00815209]


In [71]:
import joblib
joblib.dump(regressor, './deployment/tfidf-model.pkl')

['./deployment/tfidf-model.pkl']